<a href="https://colab.research.google.com/github/Sayandip2023/Natural-Language-Processing-Projects/blob/main/Malaria.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, BatchNormalization
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
from keras.optimizers import AdamW
import tensorflow_datasets as tfds

# Set parameters
input_shape = (64, 64, 3)  # Adjusted input shape for preprocessing
dropout_rate = 0.27
num_classes = 2  # Malaria dataset has 2 classes: parasitized and uninfected
batch_size = 32
epochs = 100

# Load the Malaria dataset
train_ds, ds_info = tfds.load(
    'malaria',
    split='train',
    shuffle_files=True,
    as_supervised=True,
    with_info=True
)

# Split train dataset into train and test (validation) sets
train_ds_size = ds_info.splits['train'].num_examples
test_ds_size = int(0.2 * train_ds_size)  # 20% for test (validation)
train_ds = train_ds.take(train_ds_size)
test_ds = train_ds.skip(train_ds_size - test_ds_size)

# Define preprocessing function
def preprocess_image(image, label):
    image = tf.image.resize(image, (64, 64)) / 255.0
    return image, label

# Preprocess train dataset
train_ds = train_ds.map(lambda image, label: preprocess_image(image, label)).batch(batch_size)

# Preprocess test dataset
test_ds = test_ds.map(lambda image, label: preprocess_image(image, label)).batch(batch_size)

# Build the CNN model
model = Sequential([
    Conv2D(64, (3, 3), activation='relu', input_shape=input_shape),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Dropout(dropout_rate),

    Conv2D(128, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Dropout(dropout_rate),

    Conv2D(128, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Dropout(dropout_rate),

    Conv2D(256, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Flatten(),

    Dense(512, activation='relu'),
    BatchNormalization(),
    Dropout(dropout_rate),

    Dense(256, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),

    Dense(num_classes, activation='softmax')
])

# Compile the model with AdamW optimizer
optimizer = AdamW(learning_rate=0.001)
model.compile(optimizer=optimizer,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Define callbacks
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=0.0001)
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Fit the model
history = model.fit(
    train_ds,
    epochs=epochs,
    validation_data=test_ds,
    callbacks=[reduce_lr, early_stopping]
)

# Evaluate the model
loss, accuracy = model.evaluate(test_ds)
print(f'Test loss: {loss}')
print(f'Test accuracy: {accuracy}')


Epoch 1/100
862/862 [==============================] - 49s 51ms/step - loss: 0.4057 - accuracy: 0.8331 - val_loss: 0.2420 - val_accuracy: 0.9396 - lr: 0.0010
Epoch 2/100
862/862 [==============================] - 36s 42ms/step - loss: 0.1708 - accuracy: 0.9418 - val_loss: 0.1880 - val_accuracy: 0.9408 - lr: 0.0010
Epoch 3/100
862/862 [==============================] - 43s 50ms/step - loss: 0.1570 - accuracy: 0.9478 - val_loss: 0.1792 - val_accuracy: 0.9430 - lr: 0.0010
Epoch 4/100
862/862 [==============================] - 36s 42ms/step - loss: 0.1485 - accuracy: 0.9519 - val_loss: 0.1383 - val_accuracy: 0.9548 - lr: 0.0010
Epoch 5/100
862/862 [==============================] - 36s 41ms/step - loss: 0.1459 - accuracy: 0.9529 - val_loss: 0.2238 - val_accuracy: 0.9229 - lr: 0.0010
Epoch 6/100
862/862 [==============================] - 34s 40ms/step - loss: 0.1402 - accuracy: 0.9534 - val_loss: 0.1546 - val_accuracy: 0.9539 - lr: 0.0010
Epoch 7/100
862/862 [==============================]